In [1]:
%matplotlib inline

In [2]:
from __future__ import division,print_function

import os, json
from glob import glob
import utils; reload(utils)
import vgg16; reload(vgg16)
from utils import plots
from vgg16 import Vgg16

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from keras.preprocessing.image import *

import sys

np.set_printoptions(precision=4, linewidth=100)

Using gpu device 0: GeForce GTX 1070 (CNMeM is disabled, cuDNN 5105)
Using Theano backend.


In [5]:
path = "data/dogscats/sample/"
path = "data/dogscats/"


# As large as you can, but no larger than 64 is recommended. 
# If you have an older or cheaper GPU, you'll run out of memory, so will have to decrease this.
batch_size=64
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))

In [4]:
# Instantiate
vgg = Vgg16()

# Grab a few images at a time for training and validation.
# NB: They must be in subdirectories named based on their category
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)

# Finetune and fit to validation
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=1)

# Save model
vgg.model.save_weights('models/epoch1.h5')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/1
23000/23000 [==============================] - 239s - loss: 0.2070 - acc: 0.9591 - val_loss: 0.0928 - val_acc: 0.9785

In [6]:
vgg = Vgg16()
vgg.model.load_weights('models/epoch1.h5')

In [7]:
imags = glob(os.path.join('data/kaggle_dogscats/test/unknown/','*.jpg'))
N = int(round(len(imags)/200))

In [22]:
records = []
for i in range(1,N):
    sys.stdout.write('|')
print('|')
for n, path_img in enumerate(imags):
    probs = vgg.model.predict(
        img_to_array(load_img(path_img, target_size=[224, 224])).
        reshape(1, 3,224, 224))
    number = os.path.split(path_img)[-1][0:-4]
    records.append({'id': number, 'label': probs[0][1]})
    if n % 200 == 0:
        sys.stdout.write('|')
print('')

df = pd.DataFrame.from_records(records)
df['id'] = pd.to_numeric(df['id'])
df = df.sort_values('id')

|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||


In [23]:
df_mod = df.copy()
df_mod.ix[df_mod.label<0.025, 'label'] = 0.025
df_mod.ix[df_mod.label>0.975, 'label'] = 0.975

In [24]:
df.to_csv('submissions/submission_5.csv', index=False)
df_mod.to_csv('submissions/submission_6.csv', index=False)